In [ ]:
import os
if os.name != 'nt' :
    import resource
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from App.common import V, init_globals, loading_bar, myfigures, get_query_list, mp
from App import tab1, tab2, tab3, tab4, tab5, tab6
import dmyplant2
from dmyplant2 import cred, MyPlant, Engine, cplotdef, get_size

logfilename = 'FSM.log'
import logging

logging.basicConfig(
    filename=logfilename,
    filemode='w',
    format='%(levelname)s %(message)s',
    #level=logging.INFO
    level=logging.ERROR
)
logging.info("FSM looging Start")

### Statemachine Engine Analysis ©2022 Dieter Chvatal

In [ ]:
#########################################
# tabs
#########################################
tabs_out = widgets.Output()
tabs = widgets.Tab()
tabs.children = [tab1._tab, tab2._tab, tab3._tab, tab4._tab, tab5._tab, tab6._tab]
tabs.set_title(0,'1. select Engine')
tabs.set_title(1,'2. messages & FSM')
tabs.set_title(2,'3. Filter Results')
tabs.set_title(3,'4. Start Plots')
tabs.set_title(4,'5. settings')
tabs.set_title(5,'Playground')

#@tabs_out.capture(clear_output=False)
def tabs_cb(*args):
    if tabs.selected_index >= 4:
        return
    if tabs.selected_index == 0:
            init_globals()
            #tab1.sel.value = '-'
            #tab1.selno.value = '-'
            #tab1.tdd.value = ''
            #tab1.tdd.options = V.query_list
            #tab1.es.options = ['-']

            #tab2.tab2_out.clear_output()
            tab3.tab3_out.clear_output()
            tab4.tab4_out.clear_output()
            tab2.b_runfsm.button_style=''
            tab2.b_runfsm.disabled = True
            tab2.b_loadfsm.disabled = True
            tab2.b_loadfsm.button_style = ''
            
    if tabs.selected_index > 0:
        if tab1.selno.value == '-':
            with tab1.tab1_out:
                tab1.tab1_out.clear_output()
                print()
                print('Please select an Engine first.')
            tabs.selected_index = 0
        else:
            if V.e is None:
                with tab2.tab2_out:
                    #tab2.tab2_out.clear_output()
                    print()
                    print('Please Wait ...')
                    display(loading_bar)
                    print('contacting MyPlant ...')
                    V.e=Engine.from_fleet(mp, V.fleet.iloc[int(tab1.selno.value)])
                    logging.info(f"{V.e}")
                    tab2.tab2_out.clear_output()
            V.lfigures = myfigures(V.e)
            V.plotdef, V.vset = cplotdef(mp, V.lfigures)
            tab2.el.value = tab1.sel.value
            tab2.t1.value = pd.to_datetime(V.e['Commissioning Date'])
            #tab2.tab2_out.clear_output()

        if tab1.selno.value != '-' and ((V.fsm is None) or (len(V.fsm.starts) == 0)):
            with tab2.tab2_out:
                #tab2.tab2_out.clear_output()
                print()
                print('Please load messages.')
            tabs.selected_index = 1
        if V.fsm is not None: 
            rdf = V.fsm.starts
            if not rdf.empty:
                if tab4.sno_slider.max < len(rdf):
                    tab4.sno_slider.max = rdf.shape[0]-1

with tab1.tab1_out:
    tab1.tab1_out.clear_output()
    print()
    print('Please Lookup:')
    print(' - Site Name')
    print(' - Engine Type')
    print(' - Engine Version')
    print(' - Design Number')
    print(' - serialNumber')
    print(' - assetNumber')
tabs.observe(tabs_cb, 'selected_index')
display(widgets.VBox([tabs,tabs_out]))

In [ ]:
from App import tab4
if V.fsm is not None:
    tab4.update_fig(tab4.sno.value, plotselection=tab4.plotselection.value,plot_range=tab4.time_range.value,fsm=V.fsm, VSC=True)

In [ ]:
from pprint import pprint as pp

if os.name != 'nt':
    memory = resource.getrusage(resource.RUSAGE_SELF)
    print(f"Script Memory Usage: {memory.ru_utime:5.1f} MB")

if V.fsm is not None:
    print("\nV.fsm:")
    print("------")
    print(f"{'Size fsm in MB':>40}: {get_size(V.fsm) / (1024.0*1024.0):10.3f} MB")
    keys = list (V.fsm.__dict__.keys())
    for k in keys:
        print(f"{k:>40}: {get_size(V.fsm.__dict__[k]) / (1024.0*1024.0):10.3f} MB")

    print("\n_e:")
    print("--")
    print(f"{'Size fsm._e in MB':>40}: {get_size(V.fsm._e) / (1024.0*1024.0):10.3f} MB")
    sshkeys = list (V.fsm._e.__dict__.keys())
    for sshk in sshkeys:
        print(f"{sshk:>40}: {get_size(V.fsm._e.__dict__[sshk]) / (1024.0*1024.0):10.3f} MB")    

    print("\nstartstopHandler:")
    print("-----------------")
    print(f"{'Size fsm.startstopHandler in MB':>40}: {get_size(V.fsm.startstopHandler) / (1024.0*1024.0):10.3f} MB")
    sshkeys = list (V.fsm.startstopHandler.__dict__.keys())
    for sshk in sshkeys:
        print(f"{sshk:>40}: {get_size(V.fsm.startstopHandler.__dict__[sshk]) / (1024.0*1024.0):10.3f} MB")

In [ ]:
if V.fsm is not None:
    print("\nstartstopHandler._states:")
    print("-------------------------")
    print(f"{'Size fsm.startstopHandler._states in MB':>40}: {get_size(V.fsm.startstopHandler._states) / (1024.0*1024.0):10.3f} MB")
    sshkeys = list (V.fsm.startstopHandler._states)
    for sshk in sshkeys:
        print(f"{sshk:>40}: {get_size(V.fsm.startstopHandler._states[sshk]) / (1024.0*1024.0):10.3f} MB")

In [ ]:
#V.fsm._messages

In [ ]:
#pd.DataFrame(V.fsm.message_queue[935:937])

In [ ]:
#d = V.fsm.message_queue[936:937][0]
#d

In [ ]:
#pd.DataFrame([d])

In [ ]:
#V.fsm.message_queue[935]

In [ ]:
#d=V.fsm.message_queue[936]
#d['assetId'] = 114517
#d

In [ ]:
#pd.DataFrame(d, index=[0], dtype=object).T